In [7]:
import pandas as pd
import numpy as np
import pymorphy2
import re
from natasha import (
    Segmenter,
    MorphVocab,

    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,

    PER,
    NamesExtractor,

    Doc
)



def is_phrase(string):
    return " " in string


def is_word(string):
    pattern = re.compile("[ЁёА-я]+")
    return bool(pattern.fullmatch(string))

## Глава 2. Начинаем

В этом ноутбуке мы воспользуемся `natasha` и `pymorphy2`

In [2]:
data = pd.read_csv('./data/prepared_to_tag.csv', keep_default_na = False, na_values = [''])
data.head(15)

,id,stim,word,phrase,auto_tag
0,1,абсолютный,-,абсолютный -,NULL
1,1,абсолютный,0,абсолютный 0,NULL
2,1,абсолютный,100%,абсолютный 100%,NULL
3,1,абсолютный,100%-ный,абсолютный 100%-ный,NULL
4,1,абсолютный,max,абсолютный max,NULL
5,1,абсолютный,абсолют,абсолютный абсолют,NaN
6,1,абсолютный,авторитет,абсолютный авторитет,NaN
7,1,абсолютный,агент,абсолютный агент,NaN
8,1,абсолютный,адреналин,абсолютный адреналин,NaN
9,1,абсолютный,анализ,абсолютный анализ,NaN


Создадим колонки для определения частей речи двумя библиотеками, и сразу заполним NULL'ы и фразы.

In [3]:
tagged_cols = ["morph_stim_tag", "morph_word_tag", "nata_stim_tag", "nata_word_tag"]
for c in tagged_cols:
    data[c] = None
data.morph_word_tag = data.auto_tag
data.nata_word_tag = data.auto_tag
data = data.drop(['auto_tag'], axis = 1)
data.head(15)

,id,stim,word,phrase,morph_stim_tag,morph_word_tag,nata_stim_tag,nata_word_tag
0,1,абсолютный,-,абсолютный -,None,NULL,None,NULL
1,1,абсолютный,0,абсолютный 0,None,NULL,None,NULL
2,1,абсолютный,100%,абсолютный 100%,None,NULL,None,NULL
3,1,абсолютный,100%-ный,абсолютный 100%-ный,None,NULL,None,NULL
4,1,абсолютный,max,абсолютный max,None,NULL,None,NULL
5,1,абсолютный,абсолют,абсолютный абсолют,None,NaN,None,NaN
6,1,абсолютный,авторитет,абсолютный авторитет,None,NaN,None,NaN
7,1,абсолютный,агент,абсолютный агент,None,NaN,None,NaN
8,1,абсолютный,адреналин,абсолютный адреналин,None,NaN,None,NaN
9,1,абсолютный,анализ,абсолютный анализ,None,NaN,None,NaN


### PyMorphy

Пайморфи возвращает несколько разборов: 
* pymorphy2 возвращает все допустимые варианты разбора, но на практике обычно нужен только один вариант, правильный.
* score - это оценка P(tag|word), оценка вероятности того, что данный разбор правильный. (оценивается на основе корпуса OpenCorpora: ищутся все неоднозначные слова со снятой неоднозначностью, для каждого слова считается, сколько раз ему был сопоставлен данный тег, и на основе этих частот вычисляется условная вероятность тега (с использованием сглаживания Лапласа).

__На практике это означает, что первый разбор из тех, что возвращают методы MorphAnalyzer.parse() и MorphAnalyzer.tag(), более вероятен, чем остальные__

(с) Документация

__Вывод__: если пайморфи возвращает несколько разборов, велика вероятность, что первый самый правильный, но мы будем брать все, чтобы потом уже сравнивать, что да как.

In [26]:
# pymorphy tags: http://opencorpora.org/dict.php?act=gram
MORPH_2_NKRY = {
    "NOUN": "S",
    "ADJF": "A",
    "ADJS": "A",
    "COMP": "COMP",
    "VERB": "V",
    "INFN": "V",
    "PRTF": "V",
    "PRTS": "V",
    "GRND": "V",
    "NUMR": "NUM",
    "ADVB": "ADV",
    "NPRO": "SPRO",
    "PRED": "PRAEDIC",
    "PREP": "PR",
    "CONJ": "CONJ",
    "PRCL": "PART",
    "INTJ": "INTJ",
    "UNKN": "UNKN",
}


def morph_return_tags(word):
    tags = []
    for form in morph.parse(word):
        try:
            tag = MORPH_2_NKRY[form.tag.POS]
            #         print(tag)
            if tag not in tags:
                tags.append(tag)
        except KeyError:
            #             print(word, "|||", form, "|||", form.tag, "|||", form.tag.POS)
            continue
    if len(tags) > 0:
        return "||".join(np.unique(tags))
    else:
        return "UNKN"

Запускаем пайморфи

In [14]:
morph = pymorphy2.MorphAnalyzer()

data.morph_stim_tag = data.stim.apply(morph_return_tags)
data.loc[data.word.apply(is_word), "morph_word_tag"] = data[
    data.word.apply(is_word)
].word.apply(morph_return_tags)
data

,id,stim,word,phrase,morph_stim_tag,morph_word_tag,nata_stim_tag,nata_word_tag
0,1,абсолютный,-,абсолютный -,A,NULL,None,NULL
1,1,абсолютный,0,абсолютный 0,A,NULL,None,NULL
2,1,абсолютный,100%,абсолютный 100%,A,NULL,None,NULL
3,1,абсолютный,100%-ный,абсолютный 100%-ный,A,NULL,None,NULL
4,1,абсолютный,max,абсолютный max,A,NULL,None,NULL
...,...,...,...,...,...,...,...,...
167884,1000,ярость,шар,ярость шар,S,S,None,NaN
167885,1000,ярость,эмоции,ярость эмоции,S,S,None,NaN
167886,1000,ярость,эмоция,ярость эмоция,S,S,None,NaN
167887,1000,ярость,я,ярость я,S,SPRO,None,NaN


__quick check__

Иногда встречаются такие артефакты, про которые пайморфи ничего не знает, но мы их помечаем как UNKN:)

In [17]:
data[data.word=='хз']

,id,stim,word,phrase,morph_stim_tag,morph_word_tag,nata_stim_tag,nata_word_tag
10825,62,бюрократия,хз,бюрократия хз,S,UNKN,None,NaN
30689,180,деловитый,хз,деловитый хз,A,UNKN,None,NaN
38769,229,жалость,хз,жалость хз,PRAEDIC||S,UNKN,None,NaN
58379,349,коррупция,хз,коррупция хз,S,UNKN,None,NaN
74096,446,надменный,хз,надменный хз,A,UNKN,None,NaN
80385,483,непонимание,хз,непонимание хз,S,UNKN,None,NaN
88338,530,оптимизм,хз,оптимизм хз,S,UNKN,None,NaN
96081,574,пересуды,хз,пересуды хз,S,UNKN,None,NaN
99150,592,повеса,хз,повеса хз,S,UNKN,None,NaN
104204,620,получаться,хз,получаться хз,V,UNKN,None,NaN


In [18]:
data[:30]

,id,stim,word,phrase,morph_stim_tag,morph_word_tag,nata_stim_tag,nata_word_tag
0,1,абсолютный,-,абсолютный -,A,NULL,None,NULL
1,1,абсолютный,0,абсолютный 0,A,NULL,None,NULL
2,1,абсолютный,100%,абсолютный 100%,A,NULL,None,NULL
3,1,абсолютный,100%-ный,абсолютный 100%-ный,A,NULL,None,NULL
4,1,абсолютный,max,абсолютный max,A,NULL,None,NULL
5,1,абсолютный,абсолют,абсолютный абсолют,A,S,None,NaN
6,1,абсолютный,авторитет,абсолютный авторитет,A,S,None,NaN
7,1,абсолютный,агент,абсолютный агент,A,S,None,NaN
8,1,абсолютный,адреналин,абсолютный адреналин,A,S,None,NaN
9,1,абсолютный,анализ,абсолютный анализ,A,S,None,NaN


## Natasha

Ну и теперь не сильно сложнее запустить Наташу.

In [19]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab)

def natasha_return_tags(text):
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    pos = [t.pos for t in doc.tokens]
    assert len(pos)==2
    return ' '.join(pos)

def natasha_return_tag(text):
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    pos = [t.pos for t in doc.tokens]
    assert len(pos)==1
    return pos[0]

In [20]:
nata_tags = data[data.word.apply(is_word)].phrase.apply(natasha_return_tags)
data.loc[data.word.apply(is_word), "nata_stim_tag"] = nata_tags.apply(lambda x: x.split(' ')[0])
data.loc[data.word.apply(is_word), "nata_word_tag"] = nata_tags.apply(lambda x: x.split(' ')[1])
data.loc[~data.word.apply(is_word), "nata_stim_tag"] = data.loc[~data.word.apply(is_word), "stim"].apply(natasha_return_tag)
data[:20]

,id,stim,word,phrase,morph_stim_tag,morph_word_tag,nata_stim_tag,nata_word_tag
0,1,абсолютный,-,абсолютный -,A,NULL,ADJ,NULL
1,1,абсолютный,0,абсолютный 0,A,NULL,ADJ,NULL
2,1,абсолютный,100%,абсолютный 100%,A,NULL,ADJ,NULL
3,1,абсолютный,100%-ный,абсолютный 100%-ный,A,NULL,ADJ,NULL
4,1,абсолютный,max,абсолютный max,A,NULL,ADJ,NULL
5,1,абсолютный,абсолют,абсолютный абсолют,A,S,ADJ,NOUN
6,1,абсолютный,авторитет,абсолютный авторитет,A,S,ADJ,NOUN
7,1,абсолютный,агент,абсолютный агент,A,S,ADJ,NOUN
8,1,абсолютный,адреналин,абсолютный адреналин,A,S,ADJ,NOUN
9,1,абсолютный,анализ,абсолютный анализ,A,S,ADJ,NOUN


In [27]:
# data.nata_stim_tag.apply(lambda x: MORPH_2_NKRY[x])

In [28]:
data.to_csv('./data/natasha_pymorphy.csv', index=False)

(Если что `nata_*` - колонки, с определенными частями речи Наташей, а `morph_*` - pymorphy2).